# https://www.kaggle.com/ihelon/cassava-leaf-disease-exploratory-data-analysis

# https://www.kaggle.com/yasufuminakama/cassava-resnext50-32x4d-starter-training/

# https://www.kaggle.com/abhishek/leaf-disease-inference-using-tez

# https://www.kaggle.com/khyeh0719/pytorch-efficientnet-baseline-inference-tta

# Things to add

* [x] Data augmentation
* [ ] Offline data augmentation
* [x] TTA
* [ ] Scheduler
* [ ] Dropout before classifier
* [ ] image size 512
* [ ] Error analysis
* [ ] Model predictions as original label (soft-labelling)
* [ ] Different model architectures
* [ ] Ensemble models

In [ ]:
# !pip install pretrainedmodels
!pip install timm

In [ ]:
import json
from pathlib import Path

from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

import cv2
import albumentations

import timm
# import pretrainedmodels as pm

# Utilities

In [ ]:
import time
from contextlib import contextmanager

LOGS_PATH = Path("logs")
LOGS_PATH.mkdir(exist_ok=True)


def init_logger(log_file=LOGS_PATH / 'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


LOGGER = init_logger()


@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')


In [ ]:
label_mapping_fn = "../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"
with open(label_mapping_fn, "r") as f:
    labels = json.loads(f.read())

In [ ]:
labels

In [ ]:
train_img_dir = "../input/cassava-leaf-disease-classification/train_images/"
train_img_files = list(Path(train_img_dir).iterdir())
print("Total train images: ", len(train_img_files))

In [ ]:
train_csv_fn = "../input/cassava-leaf-disease-classification/train.csv"
df_train = pd.read_csv(train_csv_fn)

df_train["class_name"] = df_train["label"].astype(str).map(labels)

In [ ]:
df_train.head()

In [ ]:
df_train.class_name.value_counts().plot(kind="bar")

In [ ]:
BASE_IMG_DIR = Path("../input/cassava-leaf-disease-classification/train_images/")

def read_img_and_cvt_format(img_path, clr_format=cv2.COLOR_BGR2RGB):
    return cv2.cvtColor(cv2.imread(img_path), clr_format)

def visualize_batch(img_ids, labels):
    
    plt.figure(figsize=(16, 12))
    
    for idx, (img_id, label) in enumerate(zip(img_ids, labels)):
        plt.subplot(3, 3, idx + 1)
        img_fn = str(BASE_IMG_DIR / img_id)
        img = read_img_and_cvt_format(img_fn)
        plt.imshow(img)
        plt.title(f"Class: {label}", fontsize=9)
        plt.axis("off")
        
    plt.show()

In [ ]:
sampled_df = df_train.sample(9)
img_ids = sampled_df["image_id"].values
labels = sampled_df["class_name"].values

visualize_batch(img_ids, labels)

In [ ]:
def sample_with_label(df, label, sample_size=9):
    
    filtered_df = df[df["label"] == label]
    sampled_df = filtered_df.sample(sample_size)
    img_ids = sampled_df["image_id"].values
    class_names = sampled_df["class_name"].values
    
    return img_ids, class_names

In [ ]:
current_label = 0
img_ids, labels = sample_with_label(df_train, current_label)

print(f"Sample images for the class: {labels[current_label]}")
visualize_batch(img_ids, labels)

In [ ]:
current_label = 1
img_ids, labels = sample_with_label(df_train, current_label)

print(f"Sample images for the class: {labels[current_label]}")
visualize_batch(img_ids, labels)

In [ ]:
current_label = 2
img_ids, labels = sample_with_label(df_train, current_label)

print(f"Sample images for the class: {labels[current_label]}")
visualize_batch(img_ids, labels)

In [ ]:
current_label = 3
img_ids, labels = sample_with_label(df_train, current_label)

print(f"Sample images for the class: {labels[current_label]}")
visualize_batch(img_ids, labels)

In [ ]:
current_label = 4
img_ids, labels = sample_with_label(df_train, current_label)

print(f"Sample images for the class: {labels[current_label]}")
visualize_batch(img_ids, labels)

In [ ]:
ssr_aug = albumentations.ShiftScaleRotate(
    shift_limit=(-0.1, 0.1),
    scale_limit=(-0.1, 0.1),
    rotate_limit=(-180, 180),
    interpolation=0,
    border_mode=4,
    p=1.0
)

In [ ]:
random_row = df_train.sample(1, random_state=42).values[0]
random_img_id, random_label, random_class_name = random_row
single_img_path = f"../input/cassava-leaf-disease-classification/train_images/{random_img_id}"
random_img = read_img_and_cvt_format(single_img_path)

plt.title(f"Class: {random_class_name}")
plt.imshow(random_img)

In [ ]:
augmented_random_img = ssr_aug(image=random_img)["image"]

plt.imshow(augmented_random_img)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class CassavaDataset(Dataset):
    
    def __init__(self, image_paths, labels=None, transform=None):
        
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        
        img_filepath = self.image_paths[idx]
        img = read_img_and_cvt_format(img_filepath)
        if self.transform:
            img = self.transform(image=img)["image"]
        
        label = 0
        if self.labels is not None:
            label = torch.tensor(self.labels[idx]).long()
        return img, label
    

In [ ]:
BASE_IMG_DIR

In [ ]:
train_img_paths = [f"{BASE_IMG_DIR}/{img_id}" for img_id in df_train["image_id"].values]
train_dataset = CassavaDataset(image_paths=train_img_paths, 
                               labels=df_train["label"].values,
                               transform=None)

for i in range(1):
    img, label = train_dataset[i]
    
    plt.title(f"Label: {label}")
    plt.imshow(img)

plt.show()

In [ ]:
class Config:
    
    model_name = "seresnext50_32x4d"
    n_epochs = 10
    batch_size = 16
    img_size = 512
    n_classes = 5
    lr = 1e-4
    weight_decay = 1e-6
    gradient_accumulation_steps = 2
    max_grad_norm = 1000
    seed = 42
    scheduler = ""
    n_fold = 5
    train_fold = [0, 1, 2, 3, 4]
    train = True
    print_every = 100
    num_workers = 4
    tta_steps = 1
    

    
import os
import random

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=Config.seed)

In [ ]:
from sklearn.model_selection import StratifiedKFold

import torch.nn as nn

In [ ]:
# Cross Validation split.

def create_cv_split(df, n_splits, *args, **kwargs):
    
    fold = StratifiedKFold(n_splits=n_splits, *args, **kwargs)
    for idx, (train_idx, val_idx) in enumerate(fold.split(df, df["label"])):
        df.loc[val_idx, "fold"] = idx
        
    df["fold"] = df["fold"].astype(int)
    return df


In [ ]:
# df_folds = create_cv_split(df_train.copy(), 
#                 n_splits=Config.n_fold, 
#                 shuffle=True, 
#                 random_state=Config.seed)

# df_folds.groupby(["fold", "label"]).size()

In [ ]:
class Classifier(nn.Module):
    
    def __init__(self, model_name, pretrained=False):
        super(Classifier, self).__init__()
        
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, Config.n_classes)
        
    def forward(self, x):
        return self.model(x)

In [ ]:
from albumentations.pytorch import ToTensorV2
from torchvision import transforms as T

# Random augmentation for the train images from the 3rd place solution 
# in the previous competition.
# def train_transform(size):
#     return T.Compose([
#         T.RandomApply([T.RandomAffine(45, shear=15)], 0.8),
#         RandomResizedCropV2(size, scale=(0.6, 1.0), ratio=(3/5, 5/3)),
#         T.RandomHorizontalFlip(),
#         T.RandomVerticalFlip(),
#         T.ToTensor(),
#         T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#         RandomErasing(probability=0.3, sh=0.3),
#     ])

#     return T.Compose([
#             T.RandomApply([T.RandomAffine(45, shear=15)], 0.8),
#             T.RandomResizedCrop(Config.img_size, 
#                                 scale=(0.6, 1.0), 
#                                 ratio=(3/5, 5/3)),
#             T.RandomHorizontalFlip(),
#             T.RandomVerticalFlip(),
#             T.Normalize(mean=[0.485, 0.456, 0.406], 
#                         std=[0.229, 0.224, 0.225]),
#             T.ToTensor(),
#             T.RandomErasing(p=0.3)
#         ])

def _get_train_transforms_without_aug():
    return albumentations.Compose([
#         albumentations.RandomResizedCrop(
#             Config.img_size, 
#             Config.img_size
#         ),
        albumentations.CenterCrop(
            Config.img_size,
            Config.img_size
        ),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        ),
        ToTensorV2()
    ])


def get_train_transforms():
    return albumentations.Compose([
#         albumentations.RandomResizedCrop(Config.img_size, 
#                                          Config.img_size,
#                                          scale=(0.6, 1.0), 
#                                          ratio=(3/5, 5/3)),
        albumentations.CenterCrop(
            Config.img_size,
            Config.img_size
        ),
        albumentations.Transpose(),
        albumentations.HorizontalFlip(),
        albumentations.VerticalFlip(),
        albumentations.ShiftScaleRotate(),
        albumentations.HueSaturationValue(
            hue_shift_limit=0.2, 
            sat_shift_limit=0.2, 
            val_shift_limit=0.2
        ),
        albumentations.RandomBrightnessContrast(
            brightness_limit=(-0.15, 0.15), 
            contrast_limit=(-0.15, 0.15)
        ),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]),
#         albumentations.CoarseDropout(p=0.3),
        ToTensorV2()
    ])


def get_test_transforms():
    
    return albumentations.Compose([
        albumentations.Resize(Config.img_size, Config.img_size),
        albumentations.Normalize(mean=[0.485, 0.456, 0.406], 
                  std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
#     return albumentations.Compose([
#         # albumentations.Resize(Config.img_size, Config.img_size),
#         albumentations.RandomResizedCrop(Config.img_size, Config.img_size),
#         albumentations.Transpose(p=0.5),
#         albumentations.HorizontalFlip(p=0.5),
#         albumentations.VerticalFlip(p=0.5),
#         albumentations.HueSaturationValue(
#             hue_shift_limit=0.2,
#             sat_shift_limit=0.2,
#             val_shift_limit=0.2,
#             p=0.5
#         ),
#         albumentations.RandomBrightnessContrast(
#             brightness_limit=(-0.1, 0.1),
#             contrast_limit=(-0.1, 0.1),
#             p=0.5
#         ),
#         albumentations.Normalize(mean=[0.485, 0.456, 0.406], 
#                                  std=[0.229, 0.224, 0.225]),
#         ToTensorV2()
#     ])

In [ ]:
# train_transforms = get_train_transforms()

# train_img_paths = [f"{BASE_IMG_DIR}/{img_id}" for img_id in df_train["image_id"].values]

# model = Classifier(Config.model_name, pretrained=False)
# train_dataset = CassavaDataset(image_paths=train_img_paths, 
#                                labels=df_train["label"].values,
#                                transform=train_transforms)

# train_data_loader = DataLoader(train_dataset, batch_size=4, 
#                                shuffle=True, num_workers=4)

# for img, label in train_data_loader:
    
#     output = model(img)
#     print(output)
#     break

In [ ]:
# for img, label in train_data_loader:
    
#     output = model(img)
#     print(output)
#     break

# Utilities

In [ ]:
import math

class AverageMeter:
    
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
        
def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return f"{m}m {s}s"


def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / percent
    rs = es - s
    return f"{as_minutes(s)} (remain {as_minutes(rs)})"


In [ ]:
import time

In [ ]:
def train_step(model, data_loader, criterion, optimizer, epoch, scheduler, device):
    """
    There is no scheduler update currently.
    """
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    # scores = AverageMeter()
    
    model.train()
    start = end = time.time()
    # global_step = 0
    total_len = len(data_loader)
    
    for step, (images, labels) in enumerate(data_loader):
        
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        preds = model(images)
        loss = criterion(preds, labels)
        losses.update(loss.item(), batch_size)
        
        if Config.gradient_accumulation_steps > 1:
            loss = loss / Config.gradient_accumulation_steps
        
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 
                                                   Config.max_grad_norm)
        if (step + 1) % Config.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            # global_step += 1
        
        batch_time.update(time.time() - end)
        end = time.time()
        if step % Config.print_every == 0 or step == (total_len - 1):
            print(f"Epoch: [{epoch+1}][{step}/{total_len}] "
                  f"Data: {data_time.val:.3f} ({data_time.avg:.3f}) "
                  f"Batch: {batch_time.val:.3f} ({batch_time.avg:.3f}) "
                  f"Elapsed: {time_since(start, float(step + 1) / (total_len))} "
                  f"Loss: {losses.val:.5f}({losses.avg:.5f}) "
                  f"Grad: {grad_norm:.4f}" # LR: {lr:.6f}
                 )
    
    return losses.avg
            

def valid_step(model, data_loader, criterion, device):
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    
    model.eval()
    start = end = time.time()
    total_len = len(data_loader)
    predictions = []
    
    for step, (images, labels) in enumerate(data_loader):
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        
        with torch.no_grad():
            preds = model(images)
        
        loss = criterion(preds, labels)
        losses.update(loss.item(), batch_size)
        predictions.append(preds.softmax(1).cpu().numpy())
        
        if Config.gradient_accumulation_steps > 1:
            loss = loss / Config.gradient_accumulation_steps
            
        batch_time.update(time.time() - end)
        end = time.time()
        
        if step % Config.print_every == 0 or step == (total_len - 1):
            print(f"Eval: [{step}/{total_len}] "
                  f"Data: {data_time.val:.3f} ({data_time.avg:.3f}) "
                  f"Batch: {batch_time.val:.3f} ({batch_time.avg:.3f}) "
                  f"Elapsed: {time_since(start, float(step + 1) / total_len)} "
                  f"Loss: {losses.val:.5f} ({losses.avg:.5f})"
                 )
    
    predictions = np.concatenate(predictions)
    return losses.avg, predictions


def inference(model, states, data_loader, device):
    
    model.to(device)
    tk0 = tqdm(enumerate(data_loader), total=len(data_loader))
    probs = []
    
    for idx, (images, _) in tk0:
        
        images = images.to(device)
        avg_preds = []
        for state in states:
            model.load_state_dict(state["model"])
            model.eval()
            with torch.no_grad():
                preds = model(images)
            
            avg_preds.append(preds.softmax(1).cpu().numpy())
        
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    
    probs = np.concatenate(probs)
    return probs

In [ ]:
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report

device = "cuda" if torch.cuda.is_available() else "cpu"

MODELS_DIR = Path("models")
MODELS_DIR.mkdir(exist_ok=False)


def train_loop(folds, fold):
    
    LOGGER.info(f"========= fold: {fold} training ===========")
    train_indices = folds[folds["fold"] != fold].index
    valid_indices = folds[folds["fold"] == fold].index
    
    train_folds = folds.loc[train_indices].reset_index(drop=True)
    valid_folds = folds.loc[valid_indices].reset_index(drop=True)

    train_img_paths = [f"{BASE_IMG_DIR}/{img_id}" for img_id in train_folds["image_id"].values]
    valid_img_paths = [f"{BASE_IMG_DIR}/{img_id}" for img_id in valid_folds["image_id"].values]
    
    train_dataset = CassavaDataset(
        train_img_paths, 
        labels=train_folds["label"].values, 
        transform=get_train_transforms()
    )
    
    valid_dataset = CassavaDataset(
        valid_img_paths,
        labels=valid_folds["label"].values,
        transform=get_test_transforms()
    )
    
    train_data_loader = DataLoader(
        train_dataset, batch_size=Config.batch_size, 
        shuffle=True, num_workers=Config.num_workers
    )
    valid_data_loader = DataLoader(
        valid_dataset, batch_size=Config.batch_size, 
        shuffle=False, num_workers=Config.num_workers
    )
    
    model = Classifier(Config.model_name, pretrained=True)
    model.to(device)
    # amsgrad = False
    optimizer = optim.Adam(model.parameters(), 
                           lr=Config.lr, 
                           weight_decay=Config.weight_decay)
    criterion = nn.CrossEntropyLoss()
    best_score = 0.0
    best_loss = np.inf

    for epoch in range(Config.n_epochs):
        
        start_time = time.time()
        avg_epoch_loss = train_step(model, 
                                    train_data_loader, 
                                    criterion, 
                                    optimizer, 
                                    epoch, 
                                    scheduler=None, 
                                    device=device)

        avg_valid_loss, valid_preds = valid_step(model, 
                                                 valid_data_loader, 
                                                 criterion, 
                                                 device)
        valid_labels = valid_folds["label"].values
        accuracy = accuracy_score(valid_labels, valid_preds.argmax(1))
        classification_result = classification_report(valid_labels, 
                                                      valid_preds.argmax(1))
        elapsed = time.time() - start_time
        LOGGER.info(f"Epoch: {epoch+1} - avg_epoch_loss: {avg_epoch_loss:.5f} - avg_val_loss: {avg_valid_loss:.5f} - time: {elapsed:.0f}s")
        LOGGER.info(f"Epoch: {epoch+1} - Accuracy: {accuracy}")
        print(classification_result)
        
        if accuracy > best_score:
            best_score = accuracy
            LOGGER.info(f"Epoch: {epoch+1} - Save best score: {best_score:.4f} Model")
            torch.save({
                "model": model.state_dict(),
                "preds": valid_preds
            }, str(MODELS_DIR / f"{Config.model_name}_fold_{fold}_best.pth"))
            
    check_point = torch.load(str(MODELS_DIR / f"{Config.model_name}_fold_{fold}_best.pth"))
    valid_folds[[str(c) for c in range(5)]] = check_point["preds"]
    valid_folds["preds"] = check_point["preds"].argmax(1)
    return valid_folds

In [ ]:
MODELS_DIR = Path("/kaggle/input/fullmodelaugmentation08676/")

predict_model = Classifier(Config.model_name, pretrained=False)
states = [torch.load(str(MODELS_DIR / f"{Config.model_name}_fold_{fold}_best.pth"), map_location=torch.device(device)) for fold in Config.train_fold]

In [ ]:
train_img_paths = [f"{BASE_IMG_DIR}/{img_id}" for img_id in df_train["image_id"].values]
train_dataset = CassavaDataset(image_paths=train_img_paths, 
                               labels=df_train["label"].values,
                               transform=_get_train_transforms_without_aug())
train_data_loader = DataLoader(train_dataset, batch_size=128, 
                               shuffle=False, num_workers=4)

In [ ]:
train_probs = inference(predict_model, states, train_data_loader, device)

In [ ]:
gt_labels = df_train["label"].values
ohe_labels = np.zeros((gt_labels.size, gt_labels.max()+1))

ohe_labels[np.arange(gt_labels.size), gt_labels] = 1

ne_indices = np.where(gt_labels != train_probs.argmax(1))[0]
ne_indices.shape

In [ ]:
df_train[["class_0", "class_1", "class_2", "class_3", "class_4"]] = train_probs

In [ ]:
df_train["pred"] = train_probs.argmax(1)
df_train.loc[df_train.label == 4, "label"] = df_train[df_train.label == 4]["pred"]

In [ ]:
Config.n_epochs = 5

In [ ]:
MODELS_DIR = Path("models")

In [ ]:
def get_result(df):
    preds = df["preds"].values
    labels = df["label"].values
    score = accuracy_score(labels, preds)
    LOGGER.info(f"Score: {score:<.5f}")


seed_torch(Config.seed)
df_folds = create_cv_split(df_train.copy(),
                           n_splits=Config.n_fold,
                           shuffle=True,
                           random_state=Config.seed)

# df_folds = df_folds.sample(100, random_state=Config.seed)

oof_df = pd.DataFrame()

for fold in range(Config.n_fold):
    if fold == 0:
        print("Skipping first fold...")
        continue
    if fold in Config.train_fold:
        _oof_df = train_loop(df_folds, fold)
        oof_df = pd.concat([oof_df, _oof_df])
        LOGGER.info(f"====== Fold: {fold} result =======")
        get_result(_oof_df)
        
LOGGER.info("============= CV ===============")
get_result(oof_df)
oof_df.to_csv("oof_df.csv", index=False)

In [ ]:
!ls models

In [ ]:
print("selam")

In [ ]:
MODELS_DIR = Path("/kaggle/input/fullmodelaugmentation08676/")

In [ ]:
# MODELS_DIR = Path("/kaggle/input/full-model-no-augmentation/seresnext_50_32_x4d_full_model_0.8620/")

predict_model = Classifier(Config.model_name, pretrained=False)
states = [torch.load(str(MODELS_DIR / f"{Config.model_name}_fold_{fold}_best.pth"), map_location=torch.device(device)) for fold in Config.train_fold]

In [ ]:
test_df = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")

BASE_TEST_IMGS = "../input/cassava-leaf-disease-classification/test_images"

test_img_paths = [f"{BASE_TEST_IMGS}/{img_id}" for img_id in test_df.image_id.values]

test_dataset = CassavaDataset(image_paths=test_img_paths, transform=get_test_transforms())
test_data_loader = DataLoader(test_dataset, 
                             batch_size=Config.batch_size, 
                             shuffle=False)

pred_probs = inference(predict_model, states, test_data_loader, device)
labels = pred_probs.argmax(1)

test_df["label"] = labels
# test_df[["image_id", "label"]].to_csv("submission.csv", index=False)

In [ ]:
labels

In [ ]:
pred_probs

In [ ]:
test_img_paths = [f"{BASE_TEST_IMGS}/{img_id}" for img_id in test_df.image_id.values]

test_dataset = CassavaDataset(image_paths=test_img_paths, transform=get_test_transforms())
test_data_loader = DataLoader(test_dataset, 
                             batch_size=Config.batch_size, 
                             shuffle=False)

# run inference 5 times
final_preds = None
for j in range(Config.tta_steps):
    preds = inference(predict_model, states, test_data_loader, device)
    temp_preds = None
    for p in preds:
        if temp_preds is None:
            temp_preds = p
        else:
            temp_preds = np.vstack((temp_preds, p))
    if final_preds is None:
        final_preds = temp_preds
    else:
        final_preds += temp_preds
final_preds /= 5


In [ ]:
final_preds

In [ ]:
!ls -lh models

In [ ]:
!ls -lh logs

In [ ]:
from IPython.display import FileLink

In [ ]:
for file in MODELS_DIR.iterdir():
    FileLink(file)

In [ ]:
FileLink("models/seresnext50_32x4d_fold_4_best.pth")

In [ ]:
FileLink("logs/train.log")

In [ ]:
!ls /kaggle/working/models

# Plot Augmented Images

In [ ]:
def visualize_img_batch(imgs, labels, rows=3, cols=3):
    
    plt.figure(figsize=(16, 12))
    assert (rows * cols) == len(imgs), "Flat grid size must be equal to total number of images"
    for idx, (img, label) in enumerate(zip(imgs, labels)):
        plt.subplot(rows, cols, idx + 1)
        plt.imshow(img)
        plt.title(f"Class: {label}", fontsize=9)
        plt.axis("off")
        
    plt.show()
    
def predict_single_image(model, states, image, device):
    
    image = image.to(device)
    avg_preds = []
    for state in states:
        model.load_state_dict(state["model"])
        model.eval()
        with torch.no_grad():
            preds = model(image)

        avg_preds.append(preds.softmax(1).cpu().numpy())

    avg_preds = np.mean(avg_preds, axis=0)    
    return avg_preds

In [ ]:
train_img_paths = [f"{BASE_IMG_DIR}/{img_id}" for img_id in df_train["image_id"].values]
train_dataset = CassavaDataset(image_paths=train_img_paths, 
                               labels=df_train["label"].values,
                               transform=get_train_transforms())
train_data_loader = DataLoader(train_dataset, batch_size=16, 
                               shuffle=False, num_workers=4)


In [ ]:
single_img = read_img_and_cvt_format(train_img_paths[7])
plt.imshow(single_img)

In [ ]:
train_data_loader = iter(train_data_loader)

In [ ]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

imgs, img_labels = next(train_data_loader)
reshaped_imgs = imgs.numpy().transpose([0, 2, 3, 1])
reshaped_imgs = std * reshaped_imgs + mean
# reshaped_imgs = np.clip(reshaped_imgs, 0, 1)

printable_labels = [labels[str(current_label.item())] for current_label in img_labels]

In [ ]:
visualize_img_batch(reshaped_imgs, printable_labels, 4, 4)

# Error Analysis

In [ ]:
train_img_paths = [f"{BASE_IMG_DIR}/{img_id}" for img_id in df_train["image_id"].values]
train_dataset = CassavaDataset(image_paths=train_img_paths, 
                               labels=df_train["label"].values,
                               transform=_get_train_transforms_without_aug())
train_data_loader = DataLoader(train_dataset, batch_size=128, 
                               shuffle=False, num_workers=4)

train_probs = inference(predict_model, states, train_data_loader, device)

In [ ]:
gt_labels = df_train["label"].values
ohe_labels = np.zeros((gt_labels.size, gt_labels.max()+1))

ohe_labels[np.arange(gt_labels.size), gt_labels] = 1

ne_indices = np.where(gt_labels != train_probs.argmax(1))[0]
ne_indices.shape

In [ ]:
df_train[["class_0", "class_1", "class_2", "class_3", "class_4"]] = train_probs
df_train["pred"] = train_probs.argmax(1)
df_train.loc[df_train.label == 4, "label"] = df_train[df_train.label == 4]["pred"]

In [ ]:
# ((df_train.label != df_train.pred) & (df_train.label == 4)).sum()
# df_train[(df_train.label != df_train.pred) & (df_train.label == 4)]
# df_train[df_train.label != df_train.pred].label.value_counts().sort_index() / df_train.label.value_counts().sort_index()
# df_train[df_train.label == 4].pred.value_counts()
# df_train[df_train.label == 4].pred.value_counts()
# df_train[df_train.label == 4].sample(10)

In [ ]:
# single_img = read_img_and_cvt_format(str(BASE_IMG_DIR / "1870238448.jpg"))
# plt.figure(figsize=(20, 16))
# plt.imshow(single_img)